In [1]:
import numpy as np
import pandas as pd
import os
import healpy as hp
from DS_data_transformation import get_prm
%config InlineBackend.figure_format = 'retina'
from DS_Planck_Unet import west_val, east_val
from matplotlib import pyplot as plt
from DS_Planck_Unet import load_planck_model, draw_pic_with_mask, draw_pic, val_pix
from DS_healpix_fragmentation import *
from DS_data_transformation import pic2fits, normalize
from DS_detector import *
import os
from astropy.coordinates import SkyCoord
from astropy import units as u
from tqdm.notebook import tqdm
from DS_data_transformation import normalize
import seaborn as sns
from DS_models_stat import stat_orig_cats, do_all_stats, stat_orig_cats_simple
from DS_detector import rematch_cat
from matplotlib.ticker import LinearLocator
from DS_healpix_fragmentation import cut_cat_by_pix

In [2]:
max_pred_unite = np.arange(0.1, 1, 0.05)
max_pred_br = np.array(pd.read_csv('./check/brcat_recall_max_pred_pix41.csv')['max_pred'])

In [3]:
def cut_max_pred(cat, max_pred):
    df = cat.copy()
    df = df[df['max_pred'] >= max_pred]
    df.index = np.arange(len(df))
    return df

In [4]:
pixels = {'41' : [41], '38' : [38], '6' : [6], 'e' : east_val, 'w' : west_val}

In [5]:
s = '/home/rt2122/Data/gen_cats/gen_all_found_brcat_prec0.2.csv'
def get_prec(s):
    return float(s[s.find('prec')+4:-4])
get_prec(s)

0.2

In [13]:
cats_dict = {'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn.csv',
            'brcat' : '/home/rt2122/Data/SRGz/brcat.csv',
            'gen' : '/home/rt2122/Data/gen_cats/',
            'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique.csv',
            'inter' : '/home/rt2122/Data/inter_cats/inter.csv',
            'inter_sn' : '/home/rt2122/Data/inter_cats/inter_sn.csv',
            'brcat_unique': '/home/rt2122/Data/inter_cats/brcat_unique.csv'}
for pixname in tqdm(pixels):
    big_pix = pixels[pixname] 
    for det_name in cats_dict:
        det_cat = None
        cat_d = {}
        if det_name == 'gen':
            dirname = cats_dict[det_name]
            files = next(os.walk(dirname))[-1]
            cat_d = {get_prec(file) : pd.read_csv(os.path.join(dirname, file)) for file in files}
        else:
            det_cat = pd.read_csv(cats_dict[det_name])
            max_pred_vals = max_pred_unite
            if det_name in ['brcat', 'brcat_unique', 'inter_sn']:
                max_pred_vals = max_pred_br
            cat_d = {max_pred_prm : cut_max_pred(det_cat, max_pred_prm) for max_pred_prm in max_pred_vals}

        recall_df = stat_orig_cats_simple(cat_d, big_pix=big_pix, read_det_files=False, match_dist=400/3600, 
                                         spec_precision=['PSZ2(z)', 'eROSITA'])
        recall_df = recall_df.sort_index()
        recall_df.index.name='max_pred'
        if det_name in ['brcat', 'inter_sn', 'brcat_unique']:
            recall_df.index.name='s/n'
        if det_name in ['gen']:
            recall_df.index.name='mpp'
        recall_df.to_csv('./check/{}_recall_max_pred_pix{}.csv'.format(det_name, pixname))

In [15]:
cats_dict = {'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn_mpp.csv',
            'brcat' : '/home/rt2122/Data/SRGz/brcat_mpp.csv',
            'gen' : '/home/rt2122/Data/gen_cats/',
            'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique_mpp.csv',
            'inter' : '/home/rt2122/Data/inter_cats/inter_mpp.csv',
            'brcat_unique': '/home/rt2122/Data/inter_cats/brcat_unique_mpp.csv'}
for pixname in tqdm(pixels):
    big_pix = pixels[pixname] 
    for det_name in cats_dict:
        det_cat = None
        cat_d = {}
        new_cat_d = {}
        if det_name == 'gen':
            dirname = cats_dict[det_name]
            files = next(os.walk(dirname))[-1]
            cat_d = {get_prec(file) : pd.read_csv(os.path.join(dirname, file)) for file in files}
            new_cat_d = cat_d
        else:
            det_cat = pd.read_csv(cats_dict[det_name])
            max_pred_vals = max_pred_unite
            if det_name in ['brcat', 'brcat_unique', 'inter_sn']:
                max_pred_vals = max_pred_br
            cat_d = {max_pred_prm : cut_max_pred(det_cat, max_pred_prm) for max_pred_prm in max_pred_vals}
            new_cat_d = {}
            for val in cat_d:
                df = cat_d[val]
                new_cat_d[df['mpp'].min()] = df

        recall_df = stat_orig_cats_simple(new_cat_d, big_pix=big_pix, read_det_files=False, match_dist=400/3600, 
                                         spec_precision=['PSZ2(z)', 'eROSITA'])
        recall_df = recall_df.sort_index()
        recall_df.index.name='mpp'
        recall_df.to_csv('./check/mpp/{}_recall_max_pred_pix{}.csv'.format(det_name, pixname))

In [10]:
list(cats_dict.keys())

['SZcat(base)',
 'SZcat(AL)',
 'brcat',
 'gen',
 'SZcat(AL)_unique',
 'inter',
 'brcat_unique']